# Workshop Azure Databricks
## 09. AutoML
## AutoML from Azure ML integration
<br>
<img src='https://github.com/retkowsky/images/blob/master/AzureMLservicebanniere.png?raw=true'>
<br>
<br>
Azure ML Documentation : https://docs.microsoft.com/en-us/azure/machine-learning/

AutoML: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train<br>
Supported models: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train?view=azure-ml-py#supported-models

## Azure ML AutoML Databricks Installation 
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-databricks-automl-environment

In [0]:
#%pip install --upgrade --force-reinstall -r https://aka.ms/automl_linux_requirements.txt

### Check the Azure ML Core SDK Version to Validate Your Installation

In [0]:
import azureml.core
print("You are using Azure ML version", azureml.core.VERSION)

You are using Azure ML version 1.22.0

## Initialize an Azure ML Workspace
### What is an Azure ML Workspace and Why Do I Need One?

An Azure ML workspace is an Azure resource that organizes and coordinates the actions of many other Azure resources to assist in executing and sharing machine learning workflows.  In particular, an Azure ML workspace coordinates storage, databases, and compute resources providing added functionality for machine learning experimentation, operationalization, and the monitoring of operationalized models.


### What do I Need?

To create or access an Azure ML workspace, you will need to import the Azure ML library and specify following information:
* A name for your workspace. You can choose one.
* Your subscription id. Use the `id` value from the `az account show` command output above.
* The resource group name. The resource group organizes Azure resources and provides a default region for the resources in the group. The resource group will be created if it doesn't exist. Resource groups can be created and viewed in the [Azure portal](https://portal.azure.com)
* Supported regions include `eastus2`, `eastus`,`westcentralus`, `southeastasia`, `westeurope`, `australiaeast`, `westus2`, `southcentralus`, `westeurope`.

In [0]:
subscription_id = "TOBEREPLACED"
resource_group = "AMLworkshop-rg"
workspace_name = "AMLworkshop"
workspace_region = "westeurope"

## Creating a Workspace
If you already have access to an Azure ML workspace you want to use, you can skip this cell.  Otherwise, this cell will create an Azure ML workspace for you in the specified subscription, provided you have the correct permissions for the given `subscription_id`.

This will fail when:
1. The workspace already exists.
2. You do not have permission to create a workspace in the resource group.
3. You are not a subscription owner or contributor and no Azure ML workspaces have ever been created in this subscription.

If workspace creation fails for any reason other than already existing, please work with your IT administrator to provide you with the appropriate permissions or to provision the required resources.

**Note:** Creation of a new workspace can take several minutes.

In [0]:
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DD2EXWUR4 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.

https://microsoft.com/devicelogin

In [0]:
ws.get_details()

Out[4]: {'id': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/AMLworkshop-rg/providers/Microsoft.MachineLearningServices/workspaces/AMLworkshop',
 'name': 'AMLworkshop',
 'identity': {'principal_id': 'ec3d3147-5547-4237-8ab7-25f145c914b9',
 'tenant_id': '72f988bf-86f1-41af-91ab-2d7cd011db47',
 'type': 'SystemAssigned'},
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Basic',
 'workspaceid': '0e272668-09cc-42d0-958d-694cc918abf6',
 'sdkTelemetryAppInsightsKey': '6d7b2b55-6d18-4126-a3ee-251f78cfa6e5',
 'description': '',
 'friendlyName': 'AMLworkshop',
 'creationTime': '2020-11-16T10:35:26.1010460+00:00',
 'containerRegistry': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourceGroups/AMLworkshop-rg/providers/Microsoft.ContainerRegistry/registries/0e27266809cc42d0958d694cc918abf6',
 'keyVault': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/amlworkshop-rg/providers/microsoft.keyvault/vaults/amlworkshop5296527961',
 'applicationInsights': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/amlworkshop-rg/providers/microsoft.insights/components/amlworkshop7235867183',
 'storageAccount': '/subscriptions/70b8f39e-8863-49f7-b6ba-34a80799550c/resourcegroups/amlworkshop-rg/providers/microsoft.storage/storageaccounts/amlworkshop1458610383',
 'hbiWorkspace': False,
 'allowPublicAccessWhenBehindVnet': False,
 'discoveryUrl': 'https://westeurope.experiments.azureml.net/discovery',
 'notebookInfo': {'fqdn': 'ml-amlworkshop-westeurope-0e272668-09cc-42d0-958d-694cc918abf6.notebooks.azure.net',
 'resource_id': '63502f08611649efb6a3420af29476fe'}}

## Configuring Your Local Environment
You can validate that you have access to the specified workspace and write a configuration file to the default configuration location, `./aml_config/config.json`.

In [0]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

ws.write_config()

## Create an Experiment

As part of the setup you have already created an Azure ML `Workspace` object. For AutoML you will need to create an `Experiment` object, which is a named object in a `Workspace` used to run experiments.

In [0]:
import logging
import os
import random
import time
import json

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [0]:
experiment_name = 'Databricks-AutoML-Diabetes'
experiment = Experiment(ws, experiment_name)

## Load Training Data Using Dataset

Automated ML takes a `TabularDataset` as input.

You are free to use the data preparation libraries/tools of your choice to do the require preparation and once you are done, you can write it to a datastore and create a TabularDataset from it.

In [0]:
#!pip install azureml-opendatasets

In [0]:
from azureml.core import Dataset
from azureml.opendatasets import Diabetes 
diabetes = Diabetes.get_tabular_dataset() # Azure ML dataset

train_dataset = diabetes

train_df = train_dataset.to_pandas_dataframe() # Convert Azure ML dataset to Pandas dataframe
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
AGE 442 non-null int64
SEX 442 non-null int64
BMI 442 non-null float64
BP 442 non-null float64
S1 442 non-null int64
S2 442 non-null float64
S3 442 non-null float64
S4 442 non-null float64
S5 442 non-null float64
S6 442 non-null int64
Y 442 non-null int64
dtypes: float64(6), int64(5)
memory usage: 38.1 KB

In [0]:
train_df.head(20)

Out[25]:

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.00,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.00,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.00,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.00,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.00,4.2905,80,135
5,23,1,22.6,89.0,139,64.8,61.0,2.00,4.1897,68,97
6,36,2,22.0,90.0,160,99.6,50.0,3.00,3.9512,82,138
7,66,2,26.2,114.0,255,185.0,56.0,4.55,4.2485,92,63
8,60,2,32.1,83.0,179,119.4,42.0,4.00,4.4773,94,110
9,29,1,30.0,85.0,180,93.4,43.0,4.00,5.3845,88,310


In [0]:
train_df.describe()

Out[27]:

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000


In [0]:
train_df.corr()

Out[28]:

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
AGE,1.000000,0.173737,0.185085,0.335428,0.260061,0.219243,-0.075181,0.203841,0.270774,0.301731,0.187889
SEX,0.173737,1.000000,0.088161,0.241010,0.035277,0.142637,-0.379090,0.332115,0.149916,0.208133,0.043062
BMI,0.185085,0.088161,1.000000,0.395411,0.249777,0.261170,-0.366811,0.413807,0.446157,0.388680,0.586450
BP,0.335428,0.241010,0.395411,1.000000,0.242464,0.185548,-0.178762,0.257650,0.393480,0.390430,0.441482
S1,0.260061,0.035277,0.249777,0.242464,1.000000,0.896663,0.051519,0.542207,0.515503,0.325717,0.212022
S2,0.219243,0.142637,0.261170,0.185548,0.896663,1.000000,-0.196455,0.659817,0.318357,0.290600,0.174054
S3,-0.075181,-0.379090,-0.366811,-0.178762,0.051519,-0.196455,1.000000,-0.738493,-0.398577,-0.273697,-0.394789
S4,0.203841,0.332115,0.413807,0.257650,0.542207,0.659817,-0.738493,1.000000,0.617859,0.417212,0.430453
S5,0.270774,0.149916,0.446157,0.393480,0.515503,0.318357,-0.398577,0.617859,1.000000,0.464669,0.565883
S6,0.301731,0.208133,0.388680,0.390430,0.325717,0.290600,-0.273697,0.417212,0.464669,1.000000,0.382483


In [0]:
label = 'Y'

## Configure AutoML

Instantiate an `AutoMLConfig` object to specify the settings and data used to run the experiment.

In [0]:
automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_errors.log',
                             primary_metric = 'r2_score',
                             iteration_timeout_minutes = 5,
                             experiment_timeout_minutes = 20,
                             iterations = 10,
                             n_cross_validations = 3,
                             max_concurrent_iterations = 3, #change it based on number of worker nodes
                             verbosity = logging.INFO,
                             spark_context=sc, #databricks/spark related
                             training_data=train_dataset,
                             label_column_name=label)

## Train the Models

Call the `submit` method on the experiment object and pass the run configuration. Execution of local runs is synchronous. Depending on the data and the number of iterations this can run for a while.

In [0]:
automl_run = experiment.submit(automl_config, show_output = True)

Running on ADB.
No run_configuration provided, running on local with default configuration
Running an experiment on spark cluster: Databricks-AutoML-Diabetes.

Parent Run ID: AutoML_8cc7ebc6-0b7b-406b-8b39-0828b24991a5

Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE: Missing feature values imputation
STATUS: PASSED
DESCRIPTION: No feature missing values were detected in the training data.
 Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE: High cardinality feature detection
STATUS: PASSED
DESCRIPTION: Your inputs were analyzed, and no high cardinality features were detected.
 Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION PIPELINE DURATION METRIC BEST
 0 MaxAbsScaler LightGBM 0:00:18 0.4149 0.4149
 1 MaxAbsScaler RandomForest 0:00:27 0.4660 0.4660
 3 MinMaxScaler RandomForest 0:00:21 0.4634 0.4660
 4 MinMaxScaler ExtremeRandomTrees 0:00:19 0.4528 0.4660
 2 MaxAbsScaler ElasticNet 0:00:17 0.4960 0.4960
 5 StandardScalerWrapper ElasticNet 0:00:17 0.4950 0.4960
 6 MinMaxScaler RandomForest 0:00:23 0.4459 0.4960
 7 MinMaxScaler RandomForest 0:00:16 0.4447 0.4960
 8 VotingEnsemble 0:00:21 0.5034 0.5034
 9 StackEnsemble 0:00:18 0.5005 0.5034

## Explore the Results

#### Portal URL for Monitoring Runs

The following will provide a link to the web interface to explore individual run details and status. In the future we might support output displayed in the notebook.

In [0]:
displayHTML("<a href={} target='_blank'>Azure Portal: {}</a>".format(automl_run.get_portal_url(), automl_run.id))

Azure Portal: AutoML_8cc7ebc6-0b7b-406b-8b39-0828b24991a5

#### Retrieve All Child Runs after the experiment is completed (in portal)
You can also use SDK methods to fetch all the child runs and see individual metrics that we log.

In [0]:
children = list(automl_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    #print(properties)
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics

mylist = pd.DataFrame(metricslist).sort_index(1)
mylist

Out[34]:

,0,1,2,3,4,5,6,7,8,9
root_mean_squared_error,58.85,56.22,54.61,56.35,56.91,54.66,57.27,57.32,54.20,54.36
normalized_root_mean_squared_error,0.18,0.18,0.17,0.18,0.18,0.17,0.18,0.18,0.17,0.17
normalized_root_mean_squared_log_error,0.17,0.16,0.16,0.16,0.16,0.16,0.16,0.17,0.16,0.16
r2_score,0.41,0.47,0.50,0.46,0.45,0.50,0.45,0.44,0.50,0.50
root_mean_squared_log_error,0.43,0.42,0.42,0.42,0.42,0.42,0.42,0.43,0.41,0.41
explained_variance,0.41,0.47,0.50,0.46,0.45,0.50,0.45,0.45,0.50,0.50
median_absolute_error,38.67,40.47,38.59,40.64,37.40,39.24,41.03,41.17,39.02,39.79
mean_absolute_error,46.35,45.84,44.13,45.52,45.51,44.17,46.35,46.59,43.85,43.99
mean_absolute_percentage_error,39.91,40.50,39.36,39.73,39.69,39.39,40.65,41.44,38.71,38.97
normalized_mean_absolute_error,0.14,0.14,0.14,0.14,0.14,0.14,0.14,0.15,0.14,0.14


### Retrieve the Best Model

Below we select the best pipeline from our iterations. The `get_output` method on `automl_classifier` returns the best run and the fitted model for the last invocation. Overloads on `get_output` allow you to retrieve the best run and fitted model for *any* logged metric or for a particular *iteration*.

In [0]:
best_run, fitted_model = automl_run.get_output()

### Download the conda environment file
From the *best_run* download the conda environment file that was used to train the AutoML model.

In [0]:
from azureml.automl.core.shared import constants
conda_env_file_name = 'conda_env.yml'
best_run.download_file(name="outputs/conda_env_v_1_0_0.yml", output_file_path=conda_env_file_name)
with open(conda_env_file_name, "r") as conda_file:
    conda_file_contents = conda_file.read()
    print(conda_file_contents)

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
 # The python interpreter version.
 # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
 - azureml-train-automl-runtime==1.22.0
 - inference-schema
 - azureml-interpret==1.22.0
 - azureml-defaults==1.22.0
- numpy>=1.16.0,<1.19.0
- pandas==0.25.1
- scikit-learn==0.22.1
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- psutil>=5.2.2,<6.0.0
channels:
- anaconda
- conda-forge

### Download the model scoring file
From the *best_run* download the scoring file to get the predictions from the AutoML model.

In [0]:
from azureml.automl.core.shared import constants
script_file_name = 'scoring_file.py'
best_run.download_file(name="outputs/scoring_file_v_1_0_0.py", output_file_path=script_file_name)
with open(script_file_name, "r") as scoring_file:
    scoring_file_contents = scoring_file.read()
    print(scoring_file_contents)

# ---------------------------------------------------------
# Copyright (c) Microsoft Corporation. All rights reserved.
# ---------------------------------------------------------
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd
import joblib

import azureml.automl.core
from azureml.automl.core.shared import logging_utilities, log_server
from azureml.telemetry import INSTRUMENTATION_KEY

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame({"AGE": pd.Series([0], dtype="int64"), "SEX": pd.Series([0], dtype="int64"), "BMI": pd.Series([0.0], dtype="float64"), "BP": pd.Series([0.0], dtype="float64"), "S1": pd.Series([0], dtype="int64"), "S2": pd.Series([0.0], dtype="float64"), "S3": pd.Series([0.0], dtype="float64"), "S4": pd.Series([0.0], dtype="float64"), "S5": pd.Series([0.0], dtype="float64"), "S6": pd.Series([0], dtype="int64")})
output_sample = np.array([0])
try:
 log_server.enable_telemetry(INSTRUMENTATION_KEY)
 log_server.set_verbosity('INFO')
 logger = logging.getLogger('azureml.automl.core.scoring_script')
except:
 pass


def init():
 global model
 # This name is model.id of model that we want to deploy deserialize the model file back
 # into a sklearn model
 model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'model.pkl')
 path = os.path.normpath(model_path)
 path_split = path.split(os.sep)
 log_server.update_custom_dimensions({'model_name': path_split[-3], 'model_version': path_split[-2]})
 try:
 logger.info("Loading model from path.")
 model = joblib.load(model_path)
 logger.info("Loading successful.")
 except Exception as e:
 logging_utilities.log_traceback(e, logger)
 raise


@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
 try:
 result = model.predict(data)
 return json.dumps({"result": result.tolist()})
 except Exception as e:
 result = str(e)
 return json.dumps({"error": result})

## Register the Fitted Model
If neither metric nor iteration are specified in the register_model call, the iteration with the best primary metric is registered.

In [0]:
tags= {"Type": "test" , 
       "Framework" : "AutoML AzureML", 
       "Team" : "DataScience" , 
       "Country" : "France"}

In [0]:
mydescription = 'AutoML Regression Model using AutoML from AzureML and Azure Databricks'

model = automl_run.register_model(description = mydescription, tags=tags)
automl_run.model_id


Out[46]: 'AutoML8cc7ebc608'

> You can see all the results from the Azure ML Studio